# Quality of Experience (QoE) Script - for AMD architecture

**Overview**

This script is designed for automated collection of Quality of Experience (QoE) for YouTube videos.

**Main components**
- **`generate_script()`**: Generates a JavaScript for reporting QoE metrics to a specified server. The script includes functions for posting data to the server periodically and handling state and playback quality changes in a video player.
- **`class DebugWatchYouTubeVideoTask`**: A task that opens a YouTube video in a browser, watches it for a specified duration, and uses the generated script to report various QoE metrics like playback quality and state changes to collection server periodically.

In [ ]:
import os
import time

from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline, Task

from returns.pipeline import is_successful
from returns.result import Result, Success, Failure

from typing import List, Optional
import random
import subprocess
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

Most of the other code is not different from previous netunicorn examples. Let's use correct credentials for our infrastructure (in your case they could be different, if you don't have netunicorn instance in your organization - you can deploy your own locally for testing purposes, see https://netunicorn.cs.ucsb.edu/examples for details)

In [ ]:
# if you have .env file locally for storing credentials, skip otherwise
if '.env' in os.listdir():
    from dotenv import load_dotenv
    load_dotenv(".env")

NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'http://localhost:26611')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', 'test')
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', 'test')
print(NETUNICORN_ENDPOINT, NETUNICORN_LOGIN, NETUNICORN_PASSWORD)

client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
client.healthcheck()

Let's get some nodes for execution. As usual, for demonstration purposes of this notebook we will take our Raspberry Pi nodes, but if your infrastructure is different - feel free to modify the next cell.

In [ ]:
nodes = client.get_nodes()

# switch for showing our infrastructure vs you doing it locally on other nodes
if os.environ.get('NETUNICORN_ENDPOINT', 'http://localhost:26611') != 'http://localhost:26611':
    working_nodes = nodes.filter(lambda node: node.name.startswith("raspi")).take(1)
else:
    working_nodes = nodes.take(1)

working_nodes

In [ ]:
# generate the javascript code for the experiment
def generate_script(server_address, server_port, report_time):
    return f"""
"use strict";
/*jshint esversion: 9 */
/* jshint -W097 */

// default url where report server is located
const quality_change_url = "https://{server_address}:{server_port}/quality";
const state_change_url = "https://{server_address}:{server_port}/state";
const stats_url = "https://{server_address}:{server_port}/report";
const report_time = {report_time};

function postReport(url, jsonData) {{
    console.debug("Posting report to", url); // Logging URL
    console.debug("Data:", jsonData); //
    // this function sends json data to report server
    let xhr = new XMLHttpRequest();
    xhr.open("POST", url, true);
    xhr.setRequestHeader("Content-Type", "application/json; charset=utf-8");
    xhr.send(JSON.stringify(jsonData));
}}

function onStateChange(event) {{
    console.debug("State changed to:", event);
    // this function catch player state changes and report them
    postReport(
        state_change_url,
        {{
            video_id_and_cpn: player.getStatsForNerds().video_id_and_cpn,
            fraction: player.getVideoLoadedFraction(),
            current_time: player.getCurrentTime(),
            new_state: event,
        }}
    );
}}

function onPlaybackQualityChange(event) {{
    console.debug("Playback quality changed to:", event);
    // this function post quality changes
    postReport(
        quality_change_url,
        {{
            video_id_and_cpn: player.getStatsForNerds().video_id_and_cpn,
            fraction: player.getVideoLoadedFraction(),
            current_time: player.getCurrentTime(),
            new_quality: event,
        }}
    );
}}

function sendStats() {{
    console.debug("Sending stats...");
    // this function is executed every X ms and reports current statistics
    let stats_for_nerds = player.getStatsForNerds();
    stats_for_nerds.playback_fraction = player.getVideoLoadedFraction();
    stats_for_nerds.current_time = player.getCurrentTime();
    console.debug("Current stats:", stats_for_nerds); // Log current stats

    postReport(stats_url, stats_for_nerds);
}}

// wait until player is ready
while (!document.getElementById("movie_player")) {{
    (async () => {{
            console.debug("Waiting for player to be ready...");

        await new Promise(r => setTimeout(r, 100));
    }})();
}}

// get the player
let player = document.getElementById("movie_player");

console.log("Player found:", player); // Log player element found

// register callbacks on state and quality changes
player.addEventListener("onStateChange", onStateChange);
player.addEventListener("onPlaybackQualityChange", onPlaybackQualityChange);

// report stats for nerds every X ms
setInterval(sendStats, report_time);
"""

In [ ]:
# Class for begining youtube video watching
class DebugWatchYouTubeVideoTask(Task):
    def __init__(
        self, video_url, duration, quality, qoe_server_address, qoe_server_port, report_time
    ):
        super().__init__()
        self.video_url = video_url
        self.duration = duration
        self.quality = quality
        self.qoe_server_address = qoe_server_address
        self.qoe_server_port = qoe_server_port
        self.report_time = report_time

    def run(self):
        try:
            qoe_extension_path = os.path.join(".", "extensions", "qoe_extension")

            script_js_path = os.path.join(qoe_extension_path, "script.js")
            with open(script_js_path, "w") as f:
                f.write(generate_script(self.qoe_server_address, self.qoe_server_port, self.report_time))

            result = self.debug_watch(self.video_url, self.duration, self.quality, qoe_extension_path)

            return result
        except Exception as e:
            print(f"Error occurred: {e}")
            return Failure(f"Error occurred: {e}")
        
    def extract_qualities(text: str) -> List[int]:
        # Because of how youtube quality menu created
        lines = text.split("\n")[1:-1]

        nums = [int(s[: s.find("p")]) for s in lines]
        return nums


    def find_closest(self, options: List[int], goal: int) -> int:
        sorted_options = sorted(options)
        if not sorted_options:
            raise Exception("Youtube parsing error: quality menu block is empty")

        opt = 0
        for ind, opt in enumerate(sorted_options):
            if opt >= goal:
                if ind > 0 and (goal - sorted_options[ind - 1] < opt - goal):
                    return options.index(sorted_options[ind - 1])
                else:
                    return options.index(opt)
        return options.index(opt)
        
    def select_quality(self, driver: webdriver.Chrome, quality: int) -> None:
        settings = driver.find_element(
            By.CLASS_NAME, "ytp-settings-button"
        )  # .find_elements_by_class_name("ytp-settings-button")
        settings.click()
        menu = driver.find_elements(By.CLASS_NAME, "ytp-menuitem")
        menu[3].click()
        quality_menu = driver.find_element(By.CLASS_NAME, "ytp-quality-menu")
        options = self.extract_qualities(quality_menu.text)
        index_to_select = (
            self.find_closest(options, quality) + 1
        )  # Because we cutted first "go back to menu" option
        menu = driver.find_elements(By.CLASS_NAME, "ytp-menuitem")
        menu[index_to_select].click()
    
    def debug_watch(self, url: str, duration: Optional[int] = 100, quality: Optional[int] = None, Statsfornerds_Path: str = None) -> Result[str, str]:

        # Display size is random popular screen size
        display_number = random.randint(100, 500)
        xvfb_process = subprocess.Popen(
            ["Xvfb", f":{display_number}", "-screen", "0", "1920x1080x24"]
        )
        os.environ["DISPLAY"] = f":{display_number}"

        options = Options()
        
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-gpu")
        # options.add_argument("--headless")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument(f"--load-extension={Statsfornerds_Path}")

        # logPath = os.path.join(Statsfornerds_Path, 'logfile.log')
        # with open(logPath, "w") as f:
        #     pass
        # if not os.path.exists(logPath):
        #     print("Logpath not exists")
        # else:
        #     print(f"Logpath: {logPath}")
        # options.add_argument("--enable-logging")
        # options.add_argument("--v=3") 
        # options.add_argument(f"--log-path={logPath}")

        # or press space part
        options.add_argument("--autoplay-policy=no-user-gesture-required")

        driver = webdriver.Chrome(service=Service(), options=options)

        time.sleep(1)
        driver.get(url)
        # To make sure we stay on our page (make sure your ad-block extension does
        # not load itself as 0 page)
        # The problem is that we do not know when the adblock page will be opened,
        # so we have to make sure that we done our best to swithced to right window
        # and give youtube ~5 secs to load in bad cases

        pages = driver.window_handles

        i = 0
        driver.switch_to.window(pages[i])

        # If current is "not ours", we know that there is ours, so let's search
        while "youtube" not in driver.current_url and i < len(pages):
            i += 1
            driver.switch_to.window(pages[i])

        # For bad internet connection case - wait and retry 5 sec
        for s in range(5):
            try:
                driver.switch_to.window(pages[i])
                video = driver.find_element(By.ID, "movie_player")
                break
            except NoSuchElementException:
                time.sleep(1)
        else:
            driver.switch_to.window(pages[i])
            video = driver.find_element(By.ID, "movie_player")

        if not (quality is None):
            self.select_quality(driver, quality)

        # video.send_keys(Keys.SPACE)  # hits space for start if option not availible

        if duration is None:
            player_status = 1  # Suppose video playing now
            while player_status != 0:  # While not stopped - see docs
                time.sleep(2)  # Random 2s constant not to check to freq
                player_status = driver.execute_script(
                    "return document.getElementById('movie_player').getPlayerState()"
                )
            how = "End of video"
        else:
            time.sleep(duration)
            # video.send_keys(Keys.SPACE)
            how = "Time limit"

        # print("******Getting logs")
        # logs = driver.get_log('browser')
        # for log in logs:
        #     print(log)

        driver.close()
        xvfb_process.kill()

        return Success(how)

In [ ]:
# Define the url of the video
YouTube_URL = "https://www.youtube.com/watch?v=r0u5URS3VXE"

As the next step, let's assemble our pipeline. Given the video_url, watching duration(in seconds), report period(in milliseconds), video quality, server address(the address of ngrok server in this example) and server port, we start the task of YouTube video watching.

In [ ]:
pipeline = (
    Pipeline()
    .then(DebugWatchYouTubeVideoTask(video_url = YouTube_URL, duration=20, report_time=6000, quality=None, qoe_server_address="Your_ngrok_server_address", qoe_server_port=443))
)

In [ ]:
# Creating the experiment
experiment = Experiment().map(pipeline, working_nodes)
experiment

We use a predefined Docker image. This image was created by taking a base `netunicorn/chromium:latest` image and installing all the required packages.

In [ ]:
from netunicorn.base import DockerImage

for deployment in experiment:
    deployment.environment_definition = DockerImage(image='sakura61777/netunicorn_local5:latest')

In [ ]:
experiment_label = "qoe_collection_AMD"

In [ ]:
try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

client.prepare_experiment(experiment, experiment_label)

while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(2)

Verifying that everything is correct:

In [ ]:
for deployment in client.get_experiment_status(experiment_label).experiment:
    print(f"Prepared: {deployment.prepared}, error: {deployment.error}")

And starting the execution:

In [ ]:
client.start_execution(experiment_label)

while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(2)

In [ ]:
for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")
    print(report)

    result, log = report.result  # report stores results of execution and corresponding log
    
    # result is a returns.result.Result object, could be Success of Failure
    # or None is error occured during execution
    
    print(f"Result is: {type(result)}")
    print(f"Result: {result}")
    
    if isinstance(result, Result):
        data = result.unwrap() if is_successful(result) else result.failure()
        print(data)
        for key, value in data.items():
            print(f"{key}: {value}")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()